# Group Module

The Group module allows you to combine several elements of a pandapower net into one group. Various functions are available, which are then automatically applied to all elements in this group.
This tutorial shows you how to define groups and how to make of some helpful simple group functions.

To analyse the group functionality we use the CIGRE MV net.

In [1]:
import numpy as np
import pandapower as pp
from pandapower.networks import create_cigre_network_mv

net = create_cigre_network_mv(with_der="all")
net.switch["closed"] = True

## Define Groups
As examples, we define two groups, one for all elements of the second feeder of the net and one to represent a virtual power plant with sgens, loads and a storage.

In [2]:
# define a Group of a Feeder 2
feeder2buses = [12, 13, 14]
feeder2_elms = pp.get_connected_elements_dict(net, feeder2buses)
feeder2_elms["bus"] = feeder2buses
gr1_object = pp.Group(net, feeder2_elms, name="Feeder2")

# define a Group as virtual power plant
vpp_elms = {"storage": 1, "sgen": [6, 8, 9, 10, 11, 12], "load": [5, 6]}
gr2_object = pp.Group(net, vpp_elms, name="virtual power plant")

Now we can see that there are two entries in `net.group`.

In [3]:
print(net.group)

                  name object
0              Feeder2  Group
1  virtual power plant  Group


The definition of a group returns the group object. This object is stored in the `object` column of `net.group`. Here we can see, that these are realy the same:

In [4]:
print(net.group.object.at[0] == gr1_object)
# or:
print(net.group.object.at[0].compare_group(gr1_object))

True
True


A group basically is a dict of stored indices of the elements which are concluded by the group.

In [5]:
print(gr1_object.elms_dict)

{'trafo': Int64Index([1], dtype='int64'), 'load': Int64Index([8, 9, 15, 16, 17], dtype='int64'), 'line': Int64Index([10, 11, 14], dtype='int64'), 'bus': Int64Index([12, 13, 14], dtype='int64'), 'switch': Int64Index([5], dtype='int64')}


## Set `elm_col`
However, the user can manipulate the indices of the net element dataframes and also some pandapower functions, such as `pp.create_continuous_elements_index(net)`, changes the indices of the elements. As a result, by means of the indices, a group can no longer find its members.
For that reason, groups can also detect their members by a column of the elements dataframes. That can be applied directly at the group definition or later using `set_elm_col()`. Then we can see, that the group does no longer store the indices but the values of the set element column:

In [6]:
print("Initial stored elms_dict")
gr2_object.set_elm_col(net, "name")
print("\nNow elms_dict stores the names of the members:\n")
print(gr2_object.elms_dict)

Initial stored elms_dict

Now elms_dict stores the names of the members:

{'storage': Index(['Battery 2'], dtype='object', name='name'), 'sgen': Index(['PV 10', 'WKA 7', 'Residential fuel cell 1', 'CHP diesel 1',
       'Fuel cell 1', 'Residential fuel cell 2'],
      dtype='object', name='name'), 'load': Index(['Load R8', 'Load R10'], dtype='object', name='name')}


As we can see, the second group now stores the names of the members and not indices. Using `get_idx()` you can nevertheless get the indices of the group members. You can see that these are still the same as given in the definition of the group above.

In [7]:
print(gr1_object.get_idx(net, "load"))

Int64Index([8, 9, 15, 16, 17], dtype='int64')


**Attention:** Be aware that `elm_col` only works fine if there are no duplicated values in `net[elm][elm_col]` for all `elm in group_object.elms_dict.keys()`!

## Update Group
The number of members per element can easily be accessed:

In [8]:
print(gr1_object.elm_counts())

trafo     1
load      5
line      3
bus       3
switch    1
dtype: int64


If the net has changed and some of the groups members are or could be removed, one can update the stored element lists of groups:

In [9]:
net.switch.drop([5], inplace=True)  # the net is changed
gr1_object.update_elms_dict(net)  # the elms_dict of gr1_object applies to the changed net

hp.misc.groups - INFO: In switch, the number of indices has been corrected from 1 to 0.


Now the counting of the members does not include the switch anymore:

In [10]:
print(gr1_object.elm_counts())

trafo    1
load     5
line     3
bus      3
dtype: int64


## Set Values to all Group Members
The following code block shows you how to set the value to all members of a group. A specific use case of the is to set all members in or out of service. For that reason, these got explicit function names.

In [11]:
# setting a name value to group 1 members
gr1_object.set_value(net, "member of group '%s'" % net.group.name.at[0], "name")
# visualize the effect
print("The load names:\n")
print(net.load.name)

# set all elements of group 1 out of service
gr1_object.set_out_of_service(net)
pp.runpp(net)
print("\nThe bus results with Feeder 2 out of service:\n")
print(net.res_bus)  # the Feeder 2 buses should now have nan values

# and back in service...
gr1_object.set_in_service(net)
pp.runpp(net)
print("\nThe bus results with Feeder 2 back in service:\n")
print(net.res_bus)

The load names:

0                       Load R1
1                       Load R3
2                       Load R4
3                       Load R5
4                       Load R6
5                       Load R8
6                      Load R10
7                      Load R11
8     member of group 'Feeder2'
9     member of group 'Feeder2'
10                     Load CI1
11                     Load CI3
12                     Load CI7
13                     Load CI9
14                    Load CI10
15    member of group 'Feeder2'
16    member of group 'Feeder2'
17    member of group 'Feeder2'
Name: name, dtype: object

The bus results with Feeder 2 out of service:

       vm_pu  va_degree       p_mw    q_mvar
0   1.030000   0.000000 -22.818564 -8.646534
1   0.993935  -6.110560  19.839000  4.637136
2   0.976713  -6.707834   0.000000  0.000000
3   0.949406  -7.665482   0.481700  0.208882
4   0.947294  -7.750513   0.411650  0.108182
5   0.946493  -7.784148   1.264500  0.182329
6   0.947350  -7.6

## Sum Group Consumption Power
Groups can sum its complete power consumption including losses. Since the virtual power plant group predominantly consists of generation units, its active power value is negative.

In [12]:
for idx in net.group.index:
    print("Group '%s' consumes %.2f MW and %.2f Mvar." % (
        net.group.name.at[idx],
        net.group.object.at[idx].res_p_mw(net),
        net.group.object.at[idx].res_q_mvar(net)))

# a validation of Feeder 2 group values is easy since there is only one trafo and one line
# which supply the feeder:
p_val = net.res_line.p_to_mw.at[14] + net.res_trafo.p_hv_mw.at[1]
assert np.isclose(gr1_object.res_p_mw(net), p_val)

Group 'Feeder2' consumes 20.66 MW and 7.45 Mvar.
Group 'virtual power plant' consumes -0.85 MW and 0.27 Mvar.


## Append and Drop Group Members
Once defined, a group is capable to change its members:

In [13]:
p_before = gr2_object.res_p_mw(net)

# append the virtual power plant
gr2_object.append_to_group({"storage": net.storage.name.at[0]})
# drop an sgen from the virtual power plant
gr2_object.drop_from_group({"load": net.load.name.at[5]})

# validate via compare the active power consumption
assert np.isclose(p_before + net.res_storage.p_mw.at[0] - net.res_load.p_mw.at[5], gr2_object.res_p_mw(net))